# 作業 : (Kaggle)鐵達尼生存預測精簡版 
https://www.kaggle.com/c/titanic

# [作業目標]
- 試著不依賴說明, 只依照下列程式碼回答下列問題, 初步理解什麼是"特徵工程"的區塊

# [作業重點]
- 試著不依賴註解, 以之前所學, 回答下列問題

# 作業1
* 下列A~E五個程式區塊中，哪一塊是特徵工程?
>> ## C

# 作業2
* 對照程式區塊 B 與 C 的結果，請問那些欄位屬於"類別型欄位"? (回答欄位英文名稱即可) 
>> ### Pclass, Sex, Sibsp, Embarked

# 作業3
* 續上題，請問哪個欄位是"目標值"?
>> ### df_titanic_train['Survived']

In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [3]:
# 程式區塊 A
# 引入需要的套件、載入資料集以Dataframe形式呈現
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

titanic_train_link = 'https://drive.google.com/open?id=1Ss4nKvq-nFedjc7FV-KvCrPXyZ1Qzjop'
fluff, titanic_train_id = titanic_train_link.split('=')
downloaded_titanic_train = drive.CreateFile({'id':titanic_train_id}) 
downloaded_titanic_train.GetContentFile('titanic_train.csv')
df_titanic_train = pd.read_csv('titanic_train.csv')

titanic_test_link = 'https://drive.google.com/open?id=13DnK3HRIfLhsKYR5RRat9b5gCa5Rag0x'
fluff, titanic_test_id = titanic_test_link.split('=')
downloaded_titanic_test = drive.CreateFile({'id':titanic_test_id}) 
downloaded_titanic_test.GetContentFile('titanic_test.csv')
df_titanic_test = pd.read_csv('titanic_test.csv')

print(df_titanic_train.shape)
print(df_titanic_test.shape)

(891, 12)
(418, 11)


In [4]:
# 程式區塊 B
# 抽取 train_Y 及 test中的id，將 train_X, test_X 該有的資料合併成 df
train_Y = df_titanic_train['Survived']
ids = df_titanic_test['PassengerId']
df_titanic_train = df_titanic_train.drop(['PassengerId', 'Survived'] , axis=1)
df_titanic_test = df_titanic_test.drop(['PassengerId'] , axis=1)
df = pd.concat([df_titanic_train,df_titanic_test])
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
# 程式區塊 C
# 特徵工程精簡版 - 補缺值、所有欄位做LabelEncoder、數字欄位做MinMaxScaler（較簡單的方式for平衡特徵間影響力）
LEncoder = LabelEncoder()
MMEncoder = MinMaxScaler()
for c in df.columns:
    df[c] = df[c].fillna(-1)
    if df[c].dtype == 'object':
        df[c] = LEncoder.fit_transform(list(df[c].values))
    df[c] = MMEncoder.fit_transform(df[c].values.reshape(-1, 1)) # 因為結果需要是一維陣列，因此用reshape降維
df.head()

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1.0,0.118683,1.0,0.283951,0.125,0.0,0.775862,0.016072,0.000000,1.000000
1,0.0,0.218989,0.0,0.481481,0.125,0.0,0.879310,0.140813,0.575269,0.333333
2,1.0,0.400459,0.0,0.333333,0.000,0.0,0.984914,0.017387,0.000000,1.000000
3,0.0,0.323124,0.0,0.444444,0.125,0.0,0.070043,0.105390,0.381720,1.000000
4,1.0,0.016845,1.0,0.444444,0.000,0.0,0.699353,0.017630,0.000000,1.000000


In [6]:
# 程式區塊 D
train_num = train_Y.shape[0]
train_X = df[:train_num]
test_X = df[train_num:]

from sklearn.linear_model import LogisticRegression
estimator = LogisticRegression()
estimator.fit(train_X, train_Y)
pred = estimator.predict(test_X)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [0]:
# 程式區塊 E
from google.colab import files
sub = pd.DataFrame({'PassengerId': ids, 'Survived': pred})
sub.to_csv('Day_017_titanic_baseline_JiunYiYang.csv', index=False)
files.download('Day_017_titanic_baseline_JiunYiYang.csv')